In [270]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

import json
def get_keys(path):
    with open(path) as f:
        return json.load(f)
# keys = get_keys("/Users/Johnny/.secret/yelp_api.json")
# api_key = keys['api_key']
# pip install tmdbsimple #Ctrl+? this line to install tmdbsimple
import tmdbsimple as tmdb
# tmdb.API_KEY = api_key

scrape = False

import ast
import time
import http.client, urllib.request, urllib.parse, urllib.error, base64

api_key = 'ceeaacb7cf024c7485e00ef8457e42dc'


user = 'Drymander'

import pickle
from tqdm import tqdm

# pip install isodate

import isodate

In [271]:
#Standard python libraries
import pandas as pd
import seaborn as sns
# sns.set_context('talk')
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings(action='ignore') 

# Preprocessing tools
from sklearn.model_selection import train_test_split,cross_val_predict,cross_validate
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
scaler = StandardScaler()
from sklearn import metrics

# Models & Utilities
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score

# Warnings
import warnings
warnings.filterwarnings(action='ignore') 

# NLP Libraries
import nltk
import collections
# nltk.download('punkt')
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
from nltk import regexp_tokenize
import re
from nltk.corpus import stopwords
from nltk.collocations import *
from nltk import FreqDist
from nltk import word_tokenize
from nltk import ngrams
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
# !pip install wordcloud
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Pull Most Recent Match Details

This is a combination of two API calls - the first is to pull the latest match for a given player based on their gamertag.  This provides the Match ID, and the second API call pulls the data using the Match ID. 

In [272]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': api_key,
}

player = 'Drymander'

params = urllib.parse.urlencode({
    # Request parameters
    'modes': 'arena',
    'start': 0,
    'count': 1,
    'include-times': True,
})

try:
    conn = http.client.HTTPSConnection('www.haloapi.com')
    conn.request("GET", f"/stats/h5/players/{player}/matches?%s" % params, "{body}", headers)
    response = conn.getresponse()
    latest_match = json.loads(response.read())
    
    match_id = latest_match['Results'][0]['Id']['MatchId']
    
    time.sleep(1.1)
    
    conn.request("GET", f"/stats/h5/arena/matches/{match_id}?%s" % params, "{body}", headers)
    response = conn.getresponse()

    match_results = json.loads(response.read())
    conn.close()
except Exception as e:
    print(f"[Errno {0}] {1}".format(e.errno, e.strerror))

####################################

match_results

{'PlayerStats': [{'XpInfo': {'PrevSpartanRank': 148,
    'SpartanRank': 148,
    'PrevTotalXP': 15015532,
    'TotalXP': 15016211,
    'SpartanRankMatchXPScalar': 2.248,
    'PlayerTimePerformanceXPAward': 277,
    'PerformanceXP': 0,
    'PlayerRankXPAward': 25,
    'BoostAmount': 0,
    'MatchSpeedWinAmount': 0,
    'ObjectivesCompletedAmount': 0},
   'PreviousCsr': None,
   'CurrentCsr': None,
   'MeasurementMatchesLeft': 10,
   'RewardSets': [],
   'KilledOpponentDetails': [{'GamerTag': 'SixKnifeWalker', 'TotalKills': 2},
    {'GamerTag': 'OoMSRoO FniX', 'TotalKills': 1},
    {'GamerTag': 'AuntieEcoDot', 'TotalKills': 3}],
   'KilledByOpponentDetails': [{'GamerTag': 'AuntieEcoDot', 'TotalKills': 4},
    {'GamerTag': 'CommandMedal52', 'TotalKills': 1},
    {'GamerTag': 'SixKnifeWalker', 'TotalKills': 3},
    {'GamerTag': 'OoMSRoO FniX', 'TotalKills': 2}],
   'FlexibleStats': {'MedalStatCounts': [],
    'ImpulseStatCounts': [{'Id': '7cda10c8-04c4-4916-8d6e-71a426a61de8',
      'Count

# Build Match Dataframe

In [273]:
i = 0

df_match = pd.DataFrame()
columns = [
    'TeamId',
    'Gamertag',
    'SpartanRank',
    'PrevTotalXP',
]
df_match = pd.DataFrame(columns = columns)

for player in match_results['PlayerStats']:

    player_dic = {}
    # Team ID
    player_dic['TeamId'] = match_results['PlayerStats'][i]['TeamId']
    # Gamer Tag
    player_dic['Gamertag'] = match_results['PlayerStats'][i]['Player']['Gamertag']
    # Spartan Rank
    player_dic['SpartanRank'] = match_results['PlayerStats'][i]['XpInfo']['SpartanRank']
    # Previous Total XP
    player_dic['PrevTotalXP'] = match_results['PlayerStats'][i]['XpInfo']['PrevTotalXP']
    df_match = df_match.append(player_dic, ignore_index=True)
    i += 1
    
df_match

# playerteam = df_match.loc[df_match['Gamertag'] == user, 'TeamId'].values[0]

# playerteam

TeamId         Gamertag SpartanRank PrevTotalXP
0      0        Drymander         148    15015532
1      0      Wittensgirl          44      634368
2      0  Xv PIPELAYER Xv         147    11889011
3      1     OoMSRoO FniX         152    72837898
4      1     AuntieEcoDot         149    20438952
5      0   RoOkIePhEnOm10          84     2439106
6      1   CommandMedal52         148    15704146
7      1   SixKnifeWalker         151    36708272

# Pull Game Base Variant Stats for All Players in Match

Using the match details, we will compile everyone's gamertags into a list and pull more specific information for each player.  We can get statistics on every game mode (or "Game Base Variant" as it is referred to in the API).

In [274]:
player_list = str(list(df_match['Gamertag']))[1:-1]

player_list
player_list = player_list.replace(', ',',')
player_list = player_list.replace("'",'')
player_list = player_list.replace(' ','+')

player_list

'Drymander,Wittensgirl,Xv+PIPELAYER+Xv,OoMSRoO+FniX,AuntieEcoDot,RoOkIePhEnOm10,CommandMedal52,SixKnifeWalker'

In [275]:

########### Python 3.2 #############
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': str(api_key),
}

params = urllib.parse.urlencode({
    # Request parameters
#     'seasonId': '{string}',
   
})

try:
    conn = http.client.HTTPSConnection('www.haloapi.com')
    conn.request("GET", f"/stats/h5/servicerecords/arena?players={player_list}&%s" % params, "{body}", headers)
    
    response = conn.getresponse()
    data = response.read()
    players_stats = json.loads(data)
#     print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

####################################
players_stats

{'Results': [{'Id': 'Drymander',
   'ResultCode': 0,
   'Result': {'ArenaStats': {'ArenaPlaylistStats': [{'PlaylistId': '7ec8ea9d-e672-4aa1-b4c3-5d0e4d0824c9',
       'MeasurementMatchesLeft': 9,
       'HighestCsr': None,
       'Csr': None,
       'CsrPercentile': None,
       'TotalKills': 9,
       'TotalHeadshots': 0,
       'TotalWeaponDamage': 1015.8301537036896,
       'TotalShotsFired': 253,
       'TotalShotsLanded': 101,
       'WeaponWithMostKills': {'WeaponId': {'StockId': 3168248199,
         'Attachments': []},
        'TotalShotsFired': 0,
        'TotalShotsLanded': 0,
        'TotalHeadshots': 0,
        'TotalKills': 4,
        'TotalDamageDealt': 512.5044994354248,
        'TotalPossessionTime': 'PT0S'},
       'TotalMeleeKills': 2,
       'TotalMeleeDamage': 417.5044994354248,
       'TotalAssassinations': 2,
       'TotalGroundPoundKills': 0,
       'TotalGroundPoundDamage': 0.0,
       'TotalShoulderBashKills': 0,
       'TotalShoulderBashDamage': 95.0,
       'T

## Compile Individual Stats into Dataframe

In [276]:
variant_id = match_results['GameBaseVariantId']
variant_id

'a2949322-dc84-45ab-8454-cf94fb28c189'

In [277]:
vdf_columns = [
    'Gamertag',
    'TotalTimePlayed',
    'K/D',
    'Accuracy',
    'WinRate',
]

vdf = pd.DataFrame(columns = vdf_columns)

i = 0

for player in players_stats['Results']:

    
    for variant in player['Result']['ArenaStats']['ArenaGameBaseVariantStats']:
        if variant['GameBaseVariantId'] == variant_id:
            variant_stats = variant
#         else:
#             continue
            
    variant_dic = {}
    variant_dic['Gamertag'] = players_stats['Results'][i]['Id']
    variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
    variant_dic['K/D'] = variant_stats['TotalKills'] / variant_stats['TotalDeaths']
    variant_dic['Accuracy'] = variant_stats['TotalShotsLanded'] / variant_stats['TotalShotsFired']
    variant_dic['WinRate'] = variant_stats['TotalGamesWon'] / variant_stats['TotalGamesLost']
    vdf = vdf.append(variant_dic, True)
    
    i += 1
    
vdf

Gamertag  TotalTimePlayed       K/D  Accuracy   WinRate
0        Drymander       278.007420  0.875475  0.357355  1.341969
1      Wittensgirl        15.698669  0.734799  0.346783  0.873239
2  Xv PIPELAYER Xv       112.897528  0.900785  0.360279  0.820961
3     OoMSRoO FniX       310.972131  1.229818  0.385693  1.643411
4     AuntieEcoDot       141.889302  0.864368  0.399830  1.120000
5   RoOkIePhEnOm10        60.670997  0.772650  0.305874  1.190871
6   CommandMedal52       130.488255  1.090894  0.331354  0.852941
7   SixKnifeWalker       188.172140  0.876698  0.342948  0.969415

# Merge Match and Game Variant Dataframes

In [278]:
df_stats = pd.merge(df_match, vdf, how='inner', on = 'Gamertag')

df_stats.round(2)

TeamId         Gamertag SpartanRank PrevTotalXP  TotalTimePlayed   K/D  \
0      0        Drymander         148    15015532           278.01  0.88   
1      0      Wittensgirl          44      634368            15.70  0.73   
2      0  Xv PIPELAYER Xv         147    11889011           112.90  0.90   
3      1     OoMSRoO FniX         152    72837898           310.97  1.23   
4      1     AuntieEcoDot         149    20438952           141.89  0.86   
5      0   RoOkIePhEnOm10          84     2439106            60.67  0.77   
6      1   CommandMedal52         148    15704146           130.49  1.09   
7      1   SixKnifeWalker         151    36708272           188.17  0.88   

   Accuracy  WinRate  
0      0.36     1.34  
1      0.35     0.87  
2      0.36     0.82  
3      0.39     1.64  
4      0.40     1.12  
5      0.31     1.19  
6      0.33     0.85  
7      0.34     0.97

In [279]:
# player_df_spartanrank = player_df.copy()

row_count = 0

for match in tqdm(match_details):
# for match in tqdm(mds):
    
#     try:
        data = match
        df_match = pd.DataFrame()
        columns = [
            'TeamId',
            'Gamertag',
            'SpartanRank',
            'PrevTotalXP',
        ]
        df_match = pd.DataFrame(columns = columns)
        i = 0

        if len(data['PlayerStats']) != 8:
            continue

        for player in data['PlayerStats']:

            player_dic = {}
            # Team ID
            player_dic['TeamId'] = data['PlayerStats'][i]['TeamId']
            if player_dic['TeamId'] not in (0, 1):
                continue
            # Gamer Tag
            player_dic['Gamertag'] = data['PlayerStats'][i]['Player']['Gamertag']
            # Spartan Rank
            player_dic['SpartanRank'] = data['PlayerStats'][i]['XpInfo']['SpartanRank']
            # Previous Total XP
            player_dic['PrevTotalXP'] = data['PlayerStats'][i]['XpInfo']['PrevTotalXP']
            df_match = df_match.append(player_dic, ignore_index=True)
            i += 1

            playerteam = df_match.loc[df_match['Gamertag'] == user, 'TeamId'].values[0]

            if data['TeamStats'][0]['Rank'] == 1:
                winner = 0
            else: winner = 1

            if winner == playerteam:
                player_win = 1
            else:
                player_win = 0
            
        player_df = df_match.loc[df_match['TeamId'] == playerteam]
        enemy_df = df_match.loc[df_match['TeamId'] != playerteam]

        row_list = []
        row_list.append(match['MatchId'])
        row_list.append(player_win)
        for row in range(len(player_df)):
            row_list.extend(list(player_df.iloc[row].values[1:]))
        for row in range(len(enemy_df)):
            row_list.extend(list(enemy_df.iloc[row].values[1:]))

        match_details_df.loc[row_count] = row_list
#         print(row_list)
        row_count += 1
 
#     except:
#         continue
    
match_details_df

NameError: name 'match_details' is not defined

# Unused Code

In [84]:
for variant in players_stats['Results'][0]['Result']['ArenaStats']['ArenaGameBaseVariantStats']:
    if variant['GameBaseVariantId'] == variant_id:
        variant_stats = variant
    else:
        continue
        
variant_stats['TotalKills']

37596

### All stats

In [115]:
# pip install isodate

import isodate

variant_dic = {}
variant_dic['Gamertag'] = players_stats['Results'][0]['Id']
variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
variant_dic['K/D'] = variant_stats['TotalKills'] / variant_stats['TotalDeaths']
variant_dic['Accuracy'] = variant_stats['TotalShotsLanded'] / variant_stats['TotalShotsFired']
variant_dic['WinRate'] = variant_stats['TotalGamesWon'] / variant_stats['TotalGamesLost']

variant_dic

{'Gamertag': 'Drymander',
 'TotalTimePlayed': 276.90485663388887,
 'K/D': 0.8751600363136944,
 'Accuracy': 0.35764911813780187,
 'WinRate': 1.339688041594454}

In [118]:
vdf.append(variant_dic, ignore_index=True)

Accuracy   Gamertag      K/D  TotalTimePlayed   WinRate
0  0.357649  Drymander  0.87516       276.904857  1.339688

TeamId         Gamertag SpartanRank PrevTotalXP
0      1        Drymander         148    14957176
1      1    GrapeSmuggler         149    22037142
2      1           Swaski         150    29628543
3      1     KnightSmooke          53      991418
4      0      MamaRifle42          76     1900074
5      0         Alan5111         139     6267978
6      0     CristianPz22          94     2840640
7      0  CallMeChief2488         146    10010389

In [ ]:
variant_dic['TotalHeadshots']= variant_stats['TotalHeadshots']
variant_dic['TotalWeaponDamage']= variant_stats['TotalWeaponDamage']
variant_dic['TotalShotsFired']= variant_stats['TotalShotsFired']
variant_dic['TotalShotsLanded']= variant_stats['TotalShotsLanded']
variant_dic['TotalMeleeKills']= variant_stats['TotalMeleeKills']
variant_dic['TotalMeleeDamage']= variant_stats['TotalMeleeDamage']
variant_dic['TotalAssassinations']= variant_stats['TotalAssassinations']
variant_dic['TotalGroundPoundKills']= variant_stats['TotalGroundPoundKills']
variant_dic['TotalGroundPoundDamage']= variant_stats['TotalGroundPoundDamage']
variant_dic['TotalShoulderBashKills']= variant_stats['TotalShoulderBashKills']
variant_dic['TotalShoulderBashDamage']= variant_stats['TotalShoulderBashDamage']
variant_dic['TotalGrenadeDamage']= variant_stats['TotalGrenadeDamage']
variant_dic['TotalPowerWeaponKills']= variant_stats['TotalPowerWeaponKills']
variant_dic['TotalPowerWeaponDamage']= variant_stats['TotalPowerWeaponDamage']
variant_dic['TotalPowerWeaponGrabs']= variant_stats['TotalPowerWeaponGrabs']
variant_dic['TotalPowerWeaponPossessionTime']= isodate.parse_duration(variant_stats['TotalPowerWeaponPossessionTime']).total_seconds() / 3600
variant_dic['TotalDeaths']= variant_stats['TotalDeaths']
variant_dic['TotalAssists']= variant_stats['TotalAssists']
variant_dic['TotalGamesCompleted']= variant_stats['TotalGamesCompleted']
variant_dic['TotalGamesWon']= variant_stats['TotalGamesWon']
variant_dic['TotalGamesLost']= variant_stats['TotalGamesLost']
variant_dic['TotalGamesTied']= variant_stats['TotalGamesTied']
variant_dic['TotalGrenadeKills']= variant_stats['TotalGrenadeKills']
variant_dic['TotalKills']= variant_stats['TotalKills']